In [0]:
dbutils.widgets.text("prm_file_date", "")
var_file_date = dbutils.widgets.get("prm_file_date")

In [0]:
display(var_file_date)

In [0]:
%sql
show databases; 
use database f1_presentation;


In [0]:

spark.sql(f"""
            CREATE OR REPLACE TEMP VIEW race_results_updated
            AS
            SELECT races.race_year, 
                constructors.name as team_name,
                drivers.driver_id,
                drivers.name as driver_name,
                races.race_id,
                results.position,
                results.points,
                11 - results.position AS calculated_points
            FROM f1_processed.results
            JOIN f1_processed.drivers ON (results.driver_id = drivers.driver_id)
            JOIN f1_processed.constructors ON (results.constructor_id = constructors.constructor_id)
            JOIN f1_processed.races ON (results.race_id = races.race_id)
            WHERE results.position <= 10
            AND results.file_date = '{var_file_date}'
""")

In [0]:
%sql 
select count(*) from race_results_updated

In [0]:
spark.sql(f"""
            CREATE TABLE IF NOT EXISTS f1_presentation.calculated_race_results
            (
            race_year INT,
            team_name STRING,
            driver_id INT,
            driver_name STRING,
            race_id INT,
            position INT,
            points INT,
            calculated_points INT,
            created_date TIMESTAMP,
            updated_date TIMESTAMP
            )
            USING DELTA
""")

In [0]:
spark.sql(f"""
        merge into f1_presentation.calculated_race_results tgt
        using race_results_updated upd
        on (tgt.driver_id = upd.driver_id AND tgt.race_id = upd.race_id) 
        when matched then
        update set tgt.position = upd.position,
        tgt.points = upd.points,
        tgt.calculated_points = upd.calculated_points,
        tgt.updated_date = current_timestamp()
        when not matched then
        insert (race_year, team_name, driver_id, driver_name, race_id, position, points, calculated_points, created_date)
        values (upd.race_year, upd.team_name, upd.driver_id, upd.driver_name, upd.race_id, upd.position, upd.points, upd.calculated_points, current_timestamp())
""")


In [0]:
%sql
SELECT * FROM f1_presentation.calculated_race_results

In [0]:
%sql
select * from race_results_updated where race_year = 2020

In [0]:
%sql
-- CREATE table f1_presentation.calculated_race_results
-- using PARQUET
-- AS
-- SELECT races.race_year, 
--        constructors.name as team_name,
--        drivers.name as driver_name,
--        results.position,
--        results.points,
--        11 - results.position AS calculated_points
-- FROM f1_processed.results
-- JOIN f1_processed.drivers ON (RESULTS.driver_id = drivers.driver_id)
-- JOIN f1_processed.constructors ON (results.constructor_id = constructors.constructor_id)
-- JOIN f1_processed.races ON (RESULTS.race_id = races.race_id)
-- where f1_processed.results.position <= 10

In [0]:
%sql
select * from f1_presentation.calculated_race_results